# Introdução

Sistemas de recomendação são ferramentas que filtram e processam informações, retornando um resultado que faça sentido para o contexto em que está sendo aplicado. Esses sistemas utilizam algoritmos específicos para oferecer sugestões personalizadas, facilitando a tomada de decisão do usuário. Neste estudo, buscamos compreender os principais algoritmos empregados em sistemas de recomendação e avaliar seu desempenho aplicando-os ao "Book-Crossing Dataset", um conjunto de dados que reúne informações sobre livros, avaliações e interações de usuários.

A partir dessa análise, pretendemos identificar os métodos mais eficazes para realizar recomendações precisas dentro deste contexto.

# Materiais e Métodos:

## Dependências

As bibliotecas necessárias para rodar os nossos testes de caso.

In [1]:
%pip install kagglehub
%pip install pandas
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_squared_error
import random
import numpy as np
import psutil
import os

/usr/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## DataSet

Nós escolhemos o Book-Crossing Dataset, que relaciona usuários, livros e avaliações, permitindo analisar padrões de leitura e preferências. Com isso, aplicaremos algoritmos de recomendação para sugerir livros personalizados, baseados em interações anteriores.

### Importa DataSet

In [3]:
path = kagglehub.dataset_download("somnambwl/bookcrossing-dataset")

books = pd.read_csv(path+"/Books.csv", sep=";", header=0)
users = pd.read_csv(path+"/Users.csv", sep=";", low_memory=False, header=0)
ratings = pd.read_csv(path+"/Ratings.csv", sep=";", low_memory=False, header=0)

### Visualização dos dados importados

In [ ]:
books.head()

In [ ]:
users.head()

In [ ]:
ratings.head()

### Filtrando dados relevantes

In [4]:
min_user_ratings = 5  # Usuarios que avaliaram no minimo 5 livros
min_book_ratings = 10  # Livros com no minimo 10 avaliações

user_counts = ratings['User-ID'].value_counts()
book_counts = ratings['ISBN'].value_counts()

filtered_ratings = ratings[
    ratings['User-ID'].isin(user_counts[user_counts >= min_user_ratings].index) &
    ratings['ISBN'].isin(book_counts[book_counts >= min_book_ratings].index)
]

### Criação da matriz de interações

In [5]:
interaction_matrix = filtered_ratings.pivot(index='User-ID', columns='ISBN', values='Rating')
interaction_matrix = interaction_matrix.fillna(0)

In [ ]:
# exibir os valores:
interaction_matrix.head()

## KNN

### Calculando a distancia euclidiana

In [6]:
distance_matrix = pairwise_distances(interaction_matrix, metric='euclidean')
distance_df = pd.DataFrame(distance_matrix, index=interaction_matrix.index, columns=interaction_matrix.index)

### Encontrando os K-vizinhos mais proximos

In [7]:
def get_nearest_neighbors(user_id, k):
    distances = distance_df.loc[user_id]
    neighbors = distances.sort_values().iloc[1:k+1]
    return neighbors.index

### Gerando as recomendações

In [9]:
def recommend_books(user_id, k, n_recommendations):
    if user_id not in interaction_matrix.index:
        raise ValueError(f"Usuário {user_id} não encontrado na matriz de interação.")

    neighbors = get_nearest_neighbors(user_id, k)
    neighbor_ratings = interaction_matrix.loc[neighbors]
    
    mean_ratings = neighbor_ratings.mean(axis=0)
    
    unread_books = interaction_matrix.loc[user_id] == 0
    recommendations = mean_ratings[unread_books].sort_values(ascending=False).head(n_recommendations)
    
    recommendations = recommendations.reset_index().merge(
        books, left_on='ISBN', right_on='ISBN'
    )
    
    return recommendations[['Title', 'ISBN', 0]].rename(columns={0: 'Predicted-Rating'})

### Testando o algoritmo

In [10]:
recommended_books = recommend_books(100004, 5, 5)
print(recommended_books)

                                               Title        ISBN  \
0  Harry Potter and the Prisoner of Azkaban (Book 3)  0439136369   
1                                     Atlantis Found  0425177173   
2                            Whirlwind (The X-Files)  0061054151   
3                                    Julie and Romeo  0609606727   
4                                        Angel Falls  0609605925   

   Predicted-Rating  
0               2.8  
1               1.8  
2               1.6  
3               0.0  
4               0.0  


## Avaliação do algoritmo

In [12]:
print(f'Memória utilizada antes: {psutil.virtual_memory().percent}%')
def train_test_split_optimized(interaction_matrix, test_fraction=0.4):
    train_matrix = {}
    test_matrix = {}

    for user_id in interaction_matrix.index:
        non_zero_items = interaction_matrix.loc[user_id][interaction_matrix.loc[user_id] > 0].index
        test_size = int(len(non_zero_items) * test_fraction)
        
        test_items = random.sample(list(non_zero_items), test_size)
        
        user_train = {item: interaction_matrix.loc[user_id, item] for item in non_zero_items if item not in test_items}
        user_test = {item: 0 for item in test_items}
        
        train_matrix[user_id] = user_train
        test_matrix[user_id] = user_test
    
    return train_matrix, test_matrix


train_matrix, test_matrix = train_test_split_optimized(interaction_matrix, test_fraction=0.4)
print(f'Memória utilizada depois: {psutil.virtual_memory().percent}%')

Memória utilizada antes: 84.9%
Memória utilizada depois: 84.8%
